In [13]:
from skimage import io, transform
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torchvision.datasets as dset
import torchvision.transforms as T
from dataLoader import OrganoidDataset
from torch.utils import data
import numpy as np
import sys
import pandas as pd

from dataLoader import OrganoidDataset
from conv_model import SimpleConvNet
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

params = {'batch_size': 100, # low for testing
          'shuffle': True, 'num_workers' : 1}
max_epochs = 20

In [3]:
use_cuda

False

In [4]:
label_path = '../data/well_summary_A1_e0891BSA_all.csv'

In [5]:
label = pd.read_csv(label_path)

In [6]:
new_columns = ['well_id','day','median_pixel_intensity','mw_area_shape','hyst1_area','hyst2_area']
label.columns = new_columns

In [7]:
def get_image_name(well_id,day_id):
    image_name = 'well'+str(well_id).zfill(4)+'_day'+str(day_id).zfill(2)+'_well.png'
    return image_name
def get_well_label(well_id):
    return str(well_id).zfill(4)

In [8]:
label['image_name'] = label[['well_id','day']].apply(lambda x: get_image_name(*x),axis=1)
label['well_label'] = label['well_id'].apply(lambda x: get_well_label(x))

In [8]:
label.head(2)

,well_id,day,median_pixel_intensity,mw_area_shape,hyst1_area,hyst2_area
0,0,0,125.0,7830,512,418
1,1,0,125.0,8265,3044,3008


In [10]:
label.query("mw_area_shape > 32000").sort_values(by=['well_id','day'])

,well_id,day,median_pixel_intensity,mw_area_shape,hyst1_area,hyst2_area,image_name,well_label
43203,3,9,133.0,33124,14317,13965,well0003_day09_well.png,0003
33605,5,7,144.0,33124,14895,14654,well0005_day07_well.png,0005
48005,5,10,132.0,33124,12929,12666,well0005_day10_well.png,0005
57605,5,12,135.0,33124,13279,12997,well0005_day12_well.png,0005
19207,7,4,132.0,33124,14855,14586,well0007_day04_well.png,0007
43223,23,9,133.0,33124,17816,17288,well0023_day09_well.png,0023
33625,25,7,144.0,33124,20824,19976,well0025_day07_well.png,0025
43225,25,9,127.0,33124,23727,23129,well0025_day09_well.png,0025
52825,25,11,128.0,33124,29729,29134,well0025_day11_well.png,0025
14430,30,3,143.0,33124,13660,13486,well0030_day03_well.png,0030


In [11]:
excluded_well_id = set(label.query("mw_area_shape >= 32000")['well_id'].tolist())

In [12]:
len(excluded_well_id)

1510

In [13]:
label.isin(excluded_well_id).shape

(67200, 8)

In [9]:
corrected_labels = label.query("mw_area_shape < 32000")

In [11]:
corrected_labels.to_csv('filtered_well_descriptions.txt')

In [16]:
day1wells = corrected_labels.query("day == 1")
day13wells = corrected_labels.query("day == 13")

In [17]:
day1wells.shape, day13wells.shape

((4721, 8), (4712, 8))

In [18]:
merged_days = day1wells.merge(day13wells, on=['well_label'],suffixes=['_1','_13'])

In [19]:
merged_days.shape

(4642, 15)

In [20]:
well_labels = merged_days['well_label']
finalSizes = merged_days['hyst2_area_13']

In [16]:
# for i in range(4800):
#     well_labels.append(str(i).zfill(4))

In [72]:
#NUM_TRAIN = 49000

In [73]:
path = '../data/CS231n_Tim_Shan_example_data/'
initial_train_set = OrganoidDataset(path2files = path, well_labels = well_labels, day_label_X = ['01']*4642, sizes = finalSizes)
# training_generator = data.DataLoader(initial_train_set,  batch_size=64, 
#                           sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

In [80]:
training_generator = data.DataLoader(initial_train_set, **params)

In [81]:
training_generator

In [82]:

# NUM_TRAIN = 49000
# loader_train = DataLoader(training_generator, batch_size=64, 
#                           sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

In [83]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

def test_flatten():
    x = torch.arange(12).view(2, 1, 3, 2)
    print('Before flattening: ', x)
    print('After flattening: ', flatten(x))

test_flatten()
class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)

Before flattening:  tensor([[[[ 0,  1],
          [ 2,  3],
          [ 4,  5]]],


        [[[ 6,  7],
          [ 8,  9],
          [10, 11]]]])
After flattening:  tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11]])


In [84]:
in_channel = 1
channel_1 = 32
channel_2 = 16
image_size = 193
output_channel = 1
learning_rate = 1e-2


model = None
optimizer = None

################################################################################
# TODO: Rewrite the 2-layer ConvNet with bias from Part III with the           #
# Sequential API.                                                              #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
model = nn.Sequential(
    nn.Conv2d(in_channels=in_channel,out_channels=channel_1,kernel_size=5,padding=2,bias=True),
    nn.ReLU(),
    nn.Conv2d(in_channels=channel_1,out_channels=channel_2,kernel_size=3,padding=1,bias=True),
    nn.ReLU(),   
    Flatten(),
    nn.Linear(channel_2*image_size*image_size, output_channel,bias=True),
)

In [85]:
model = model.to(device=device)

In [ ]:
# for epoch in range(max_epochs):
#   # Training
#   optimizer.zero_grad()
#   totalbatchMSE = 0.0
#   for local_X, local_Y in training_generator:
#     local_X, local_Y = local_X.to(device), local_Y.to(device)
#     Y_hat = model.run_all_forward(local_X)
#     train_error = loss(Y_hat, local_Y)
#     train_error.backward()
#     optimizer.step()
#     model.eval() # set evaluation mode
#     Y_hat = model.run_all_forward(local_X)
#     train_error = loss(Y_hat, local_Y).item()
#     totalbatchMSE = totalbatchMSE + params['batch_size']*train_error/4800 # rescale train_error
#     train_error_array[epoch] = totalbatchMSE

In [12]:
def train_part34(model, optimizer, epochs=1):
    train_errors = {}
    for e in range(epochs):
        totalbatchMSE = 0.0
        for t, (x, y) in enumerate(training_generator):
            x = x.to(device=device)  # move to device, e.g. GPU
            y = y.to(device=device)
            Y_hat = model(x)
            #loss = nn.MSELoss(Y_hat,y)
            loss = F.mse_loss(Y_hat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_error = loss.item()
            totalbatchMSE = totalbatchMSE + params['batch_size']*train_error/4642.0
            train_errors[e] = totalbatchMSE
            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, train_error))
                print(totalbatchMSE)
        return train_errors
    

In [ ]:
print_every = 100
epoches = 200

optimizer =optim.Adam(model.parameters(), lr=learning_rate)

train_part34(model, optimizer,epoches)


Iteration 0, loss = 600899.5625
12944.841932356743


# call script

In [14]:
model = SimpleConvNet()

TypeError: super(type, obj): obj must be an instance or subtype of type

In [ ]:
print_every = 100
epoches = 200

optimizer =optim.Adam(model.parameters(), lr=learning_rate)

train_part34(model, optimizer,epoches)